In [1]:
import numpy as np # linear algebra -for numeric computations
import pandas as pd # data processing -to store data as dataframes 
import matplotlib.pyplot as plt # data visualization 
%matplotlib inline
import seaborn as sns # data visualization 

In [2]:
data = pd.read_csv('prosperLoanData.csv')
data.head(7)

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20
5,0F05359734824199381F61D,1074836,2013-12-14 08:26:37.093000000,NaN,60,Current,NaN,0.15425,0.1314,0.1214,...,-25.33,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
6,0F0A3576754255009D63151,750899,2013-04-12 09:52:56.147000000,NaN,36,Current,NaN,0.31032,0.2712,0.2612,...,-22.95,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1


In [3]:
# parsing Dates
data['ListingCreationDate'] = pd.to_datetime(data['ListingCreationDate'])
data['ClosedDate'] = pd.to_datetime(data['ClosedDate'])
data['DateCreditPulled'] = pd.to_datetime(data['DateCreditPulled'])
data['FirstRecordedCreditLine'] = pd.to_datetime(data['FirstRecordedCreditLine'])
data['LoanOriginationDate'] = pd.to_datetime(data['LoanOriginationDate'])


In [4]:
#  removing any feature with more than 75% of missing values.
data_with_less_missing_values = data.dropna(thresh=data.shape[0] * 0.25, axis=1)
data_with_less_missing_values.shape

(113937, 71)

In [5]:
# removing loan samples with have more than 20% of missing values
data_with_less_missing_values = data_with_less_missing_values.dropna(thresh=data.shape[1] * 0.80, axis=0).reset_index(drop=True)
data_with_less_missing_values.shape

(103360, 71)

In [6]:
cat_cols = [name for name in data_with_less_missing_values 
                        if data_with_less_missing_values[name].dtype in ["object", "bool" ]]
numerical_cols = [name for name in data_with_less_missing_values.columns
                      if data_with_less_missing_values[name].dtype in ['int64', 'float64', 'datetime64[ns]']]
cat_data = data_with_less_missing_values.drop(axis=1, columns=numerical_cols)
num_data = data_with_less_missing_values.drop(axis=1, columns=cat_cols)


In [7]:
cat_data.Occupation = cat_data.Occupation.fillna(cat_data.Occupation.mode().iloc[0])

In [8]:
cat_data = cat_data.drop(axis=1, columns=['ProsperRating (Alpha)'])

In [9]:

# all missing values in the CreditGrade column represents the rating value 0
# fill in nan values with letter Z and then use OrdinalEncoder to convert it to numerical values
cat_data.CreditGrade = cat_data.CreditGrade.fillna("Z")
from sklearn.preprocessing import OrdinalEncoder
ratings = ['Z', 'HR', 'E', 'D', 'C', 'B', 'A', 'AA']
encoder = OrdinalEncoder(categories = [ratings])
cat_data[['CreditGrade']] = encoder.fit_transform(cat_data[['CreditGrade']])
cat_data.CreditGrade = cat_data.CreditGrade.astype(int)

In [10]:
for column in cat_data.columns:
    print(f'{column} : {len(data[column].unique())}')

ListingKey : 113066
CreditGrade : 9
LoanStatus : 12
BorrowerState : 52
Occupation : 68
EmploymentStatus : 9
IsBorrowerHomeowner : 2
CurrentlyInGroup : 2
IncomeRange : 8
IncomeVerifiable : 2
LoanKey : 113066
LoanOriginationQuarter : 33
MemberKey : 90831


In [11]:
cat_data = cat_data.drop(columns=['LoanKey','MemberKey','ListingKey'],axis=1)

In [12]:
for column in cat_data.columns:
    print(f'{column} : {len(data[column].unique())}')

CreditGrade : 9
LoanStatus : 12
BorrowerState : 52
Occupation : 68
EmploymentStatus : 9
IsBorrowerHomeowner : 2
CurrentlyInGroup : 2
IncomeRange : 8
IncomeVerifiable : 2
LoanOriginationQuarter : 33


In [13]:
cat_data.isna().sum()

CreditGrade               0
LoanStatus                0
BorrowerState             0
Occupation                0
EmploymentStatus          0
IsBorrowerHomeowner       0
CurrentlyInGroup          0
IncomeRange               0
IncomeVerifiable          0
LoanOriginationQuarter    0
dtype: int64

In [14]:
num_data.isna().sum()

ListingNumber                             0
ListingCreationDate                       0
Term                                      0
ClosedDate                            58848
BorrowerAPR                               0
BorrowerRate                              0
LenderYield                               0
EstimatedEffectiveYield               18507
EstimatedLoss                         18507
EstimatedReturn                       18507
ProsperRating (numeric)               18507
ProsperScore                          18507
ListingCategory (numeric)                 0
EmploymentStatusDuration                 19
DateCreditPulled                          0
CreditScoreRangeLower                     0
CreditScoreRangeUpper                     0
FirstRecordedCreditLine                   0
CurrentCreditLines                        0
OpenCreditLines                           0
TotalCreditLinespast7years                0
OpenRevolvingAccounts                     0
OpenRevolvingMonthlyPayment     

In [15]:
num_data['DebtToIncomeRatio'].fillna(value=num_data['DebtToIncomeRatio'].mean(), inplace=True)

In [16]:
plt.figure(figsize=(45,10))
num_data.EmploymentStatusDuration = num_data.EmploymentStatusDuration.fillna(num_data.EmploymentStatusDuration.mode().iloc[0])

<Figure size 3240x720 with 0 Axes>

In [17]:
num_data.ClosedDate.fillna(value='Not Closed', inplace=True)

In [18]:
num_data.isna().sum()

ListingNumber                             0
ListingCreationDate                       0
Term                                      0
ClosedDate                                0
BorrowerAPR                               0
BorrowerRate                              0
LenderYield                               0
EstimatedEffectiveYield               18507
EstimatedLoss                         18507
EstimatedReturn                       18507
ProsperRating (numeric)               18507
ProsperScore                          18507
ListingCategory (numeric)                 0
EmploymentStatusDuration                  0
DateCreditPulled                          0
CreditScoreRangeLower                     0
CreditScoreRangeUpper                     0
FirstRecordedCreditLine                   0
CurrentCreditLines                        0
OpenCreditLines                           0
TotalCreditLinespast7years                0
OpenRevolvingAccounts                     0
OpenRevolvingMonthlyPayment     

# added new

In [19]:
num_data['EstimatedEffectiveYield'].fillna(value=num_data['EstimatedEffectiveYield'].median(), inplace=True)
num_data['EstimatedLoss'].fillna(value=num_data['EstimatedLoss'].median(), inplace=True)
num_data['EstimatedReturn'].fillna(value=num_data['EstimatedReturn'].median(), inplace=True)
num_data['ProsperRating (numeric)'].fillna(value=num_data['ProsperRating (numeric)'].median(), inplace=True)
num_data['ProsperScore'].fillna(value=num_data['ProsperScore'].median(), inplace=True)

In [20]:
num_data.isna().sum()

ListingNumber                         0
ListingCreationDate                   0
Term                                  0
ClosedDate                            0
BorrowerAPR                           0
BorrowerRate                          0
LenderYield                           0
EstimatedEffectiveYield               0
EstimatedLoss                         0
EstimatedReturn                       0
ProsperRating (numeric)               0
ProsperScore                          0
ListingCategory (numeric)             0
EmploymentStatusDuration              0
DateCreditPulled                      0
CreditScoreRangeLower                 0
CreditScoreRangeUpper                 0
FirstRecordedCreditLine               0
CurrentCreditLines                    0
OpenCreditLines                       0
TotalCreditLinespast7years            0
OpenRevolvingAccounts                 0
OpenRevolvingMonthlyPayment           0
InquiriesLast6Months                  0
TotalInquiries                        0


In [21]:
num_data.select_dtypes(include=('object'))

,ClosedDate
0,2009-08-14 00:00:00
1,Not Closed
2,Not Closed
3,Not Closed
4,Not Closed
...,...
103355,Not Closed
103356,Not Closed
103357,Not Closed
103358,2013-08-13 00:00:00


In [22]:
num_data['ClosedDate'] = num_data['ClosedDate'].fillna(0)
num_data['ClosedDate'] = num_data['ClosedDate'].apply(lambda x:1 if x!='Not Closed' else 0)

In [23]:
num_data.ClosedDate

0         1
1         1
2         1
3         1
4         1
         ..
103355    1
103356    1
103357    1
103358    1
103359    1
Name: ClosedDate, Length: 103360, dtype: int64

In [24]:
modified_data = num_data.join(cat_data)

In [25]:
modified_data.select_dtypes(include=('bool')).columns

Index(['IsBorrowerHomeowner', 'CurrentlyInGroup', 'IncomeVerifiable'], dtype='object')

In [26]:
bools = ['IsBorrowerHomeowner', 'CurrentlyInGroup', 'IncomeVerifiable']
for i in bools:
    modified_data[i] = modified_data[i].apply(lambda x:1 if x else 0)

In [27]:
modified_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103360 entries, 0 to 103359
Data columns (total 67 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   ListingNumber                       103360 non-null  int64         
 1   ListingCreationDate                 103360 non-null  datetime64[ns]
 2   Term                                103360 non-null  int64         
 3   ClosedDate                          103360 non-null  int64         
 4   BorrowerAPR                         103360 non-null  float64       
 5   BorrowerRate                        103360 non-null  float64       
 6   LenderYield                         103360 non-null  float64       
 7   EstimatedEffectiveYield             103360 non-null  float64       
 8   EstimatedLoss                       103360 non-null  float64       
 9   EstimatedReturn                     103360 non-null  float64       
 10  ProsperR

In [28]:
modified_data.to_csv('cleaned.csv',index=True)
